In [11]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [12]:
html = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup=BeautifulSoup(html.content,features='html.parser')
table=soup.find('table')
rows=table.findAll('tr')
table_data={}
ix=0
for r in rows:
    row_data=r.findAll('td')
    if row_data:
        #assume that there are only 3 columns
        table_data.update({
            ix:[
                row_data[0].text.replace("\n",""), 
                row_data[1].text.replace("\n",""), 
                row_data[2].text.replace("/", ",").replace("\n", ""),
            ]
        })
        ix+=1

In [13]:
columns=['Postal Code', 'Borough', 'Neighborhoods']

In [14]:
df = pd.DataFrame.from_dict(data=table_data, orient='index', columns=columns)

In [15]:
df.head()

,Postal Code,Borough,Neighborhoods
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


In [16]:
new_data={}
for index, row in df.iterrows():
    if row['Borough']=='Not assigned':
        continue
    
    if row['Neighborhoods']=='Not assigned':
        row['Neighborhoods']=row['Borough']
        
    if row['Postal Code'] in new_data.keys():
        if row['Neighborhoods'] not in new_data[row['Postal Code']][2]:
            row['Neighborhoods'] = new_data[row['Postal Code']][2] + " , " + row['Neighborhoods']
        
    new_data.update({
        row['Postal Code']: [row[0],row[1],row[2]]
    })
    
# steps to create dictionary for the DataFrame
new_data_new={}
xcount=0
for i in new_data.keys():
    new_data_new.update({
        xcount: new_data[i]
    })
    xcount+=1
new_df=pd.DataFrame.from_dict(data=new_data_new, orient='index', columns=columns)

In [17]:
new_df.head()

,Postal Code,Borough,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [18]:
new_df.shape

(103, 3)

In [19]:
# Import Coordinates from CSV

In [20]:
latlng_coords=pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')

In [21]:
latlng_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
latlng_coords.shape

(103, 3)

In [23]:
combined_df=pd.merge(new_df,latlng_coords,on='Postal Code')

In [24]:
combined_df.head()

,Postal Code,Borough,Neighborhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
